In [ ]:
import pandas as pd
import nltk
import re

nltk.download('wordnet')
from collections import Counter

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

df = pd.read_csv('cleaned_data.csv')
print(df.head())

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cling\AppData\Roaming\nltk_data...


   Unnamed: 0  OBSRVTN_NB                                             PNT_NM  \
0           0      330560  Did you recognize additional slip, trip, or fa...   
1           1      164330                        Vehicle Operating Condition   
2           2      265239                       Suspended Load/Overhead Work   
3           3      404438                                    PPE - Workforce   
4           4       83172                              Climbing - Procedures   

                              PNT_ATRISKFOLWUPNTS_TX      datetime_parsed  \
0                                                NaN  2023-03-15 11:01:00   
1                                                NaN  2019-07-09 10:00:00   
2  Coached employee that once he had the culverts...  2022-08-01 10:15:00   
3                                                NaN  2023-10-18 12:06:00   
4                                                NaN  2021-03-03 11:00:00   

         date      time                               cl

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cling\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\cling\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cling\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
subcategory_keywords = {
    # Gravity subcategories
    "Suspended Load": ['suspended', 'heavy', '500 lbs', 'hoisting', 'load', 'lift', 'high', 'down to', 'drop zone'],
    "Fall from Elevation": ['cliff', 'fell', 'fall from', 'ladder', 'scaffold'],
    
    # Motion subcategories
    "Vehicle Collision": ['vehicle', 'collision', 'truck', 'car', 'crashed', 'hit by', 'driver', 'mph'],
    "Mobile Equipment": ['forklift', 'truck', ' vehicle', 'mobile', 'equipment'],
    
    # Mechanical subcategories
    "Rotating Equipment": ['rotating', 'turn', 'machinery', 'moving'],
    
    # Electrical subcategories
    "Electrical Contact": ['electric', 'shock', 'contact', 'volts', 'wiring', 'zap', 'voltage', 'electricity'],
    "Arc Flash": ['arc', 'arch', 'flash', 'electrical explosion', 'electric'],

    # Temperature subcategories
    "High Temperature": ['degrees', 'burn', 'steam'],
    "Steam": ['steam', 'pressure', 'gas', 'compressed'],
    "Fire": ['degree', 'a fire', 'flame', 'burn', 'fuel', 'hot', 'oxygen'],

    # Pressure subcategories
    "Explosion": ['explode', 'pressure', 'blew up', 'blow up', 'implode'],
    "Excavation": ['soil', 'trench', 'excavat', 'pressure', 'buried', 'bury', 'deep'],

    # Chemical/Radiation
    "Chemical" : ['chem', 'radiation', 'radiated', 'expose', 'toxic']
}


In [ ]:
def assign_subcategory(description):
    description = description.lower()
    matched_subcategories = []

    for subcategory, keywords in subcategory_keywords.items():
        if any(keyword in description for keyword in keywords):
            matched_subcategories.append(subcategory)
    
    # if no subcategories matched, return "Uncategorized"
    return ', '.join(matched_subcategories) if matched_subcategories else 'Uncategorized'

# assign subcategories to each description
df['Subcategory'] = df['cleaned_atrisk_notes'].apply(assign_subcategory)
df2 = pd.read_csv('aep_data.csv')
tdf = pd.DataFrame(df['Subcategory'])
tdf['Prompt'] = df2['PNT_ATRISKNOTES_TX']
tdf.rename(columns = {'Subcategory' : 'Response'}, inplace = True)
tdf = tdf[['Prompt', 'Response']]
print(tdf.head())
tdf.to_csv('categorized_data.csv')

                                              Prompt  \
0  [NAME] was working a near by cliff that had ab...   
1  [NAME] trucks with cut out bumbers need a hitc...   
2  Employee rigged a concrete culvert to be off l...   
3  [NAME] tender was not wearing his Hardhat. I r...   
4  A drop zone was not clearly marked by the crew...   

                              Response  
0                  Fall from Elevation  
1  Vehicle Collision, Mobile Equipment  
2                       Suspended Load  
3                        Uncategorized  
4                       Suspended Load  
